In [1]:
#Set-up
import os
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
import seaborn as sns
import warnings
from IPython.display import clear_output
import nibabel as nib

import torch
import torchvision.models as models
import torch.optim as optim
import torch.nn as nn
from torch.optim import lr_scheduler
import torch.nn.functional as F
import torch.utils.data as utils
import time
import copy

In [2]:
x = torch.load("data.pt")
y = torch.load("labels.pt")
n = 536

In [3]:
USE_GPU = True

dtype = torch.float32 # we will be using float throughout this tutorial

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

print('using device:', device)

using device: cuda


In [4]:
dataset = utils.TensorDataset(x, y)

In [5]:
dataloaders = {i: torch.utils.data.DataLoader(dataset, batch_size=4,
                                             shuffle=True, num_workers=4)
              for i in ['train', 'val']}

In [6]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                scheduler.step()
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / n
            epoch_acc = running_corrects.double() / n

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [7]:
model_ft = models.resnet18(pretrained=True)

num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, 13)

model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [8]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=25)

Epoch 0/24
----------
train Loss: 2.7181 Acc: 0.1175
val Loss: 2.2047 Acc: 0.2463

Epoch 1/24
----------
train Loss: 2.3324 Acc: 0.2183
val Loss: 1.9869 Acc: 0.3097

Epoch 2/24
----------
train Loss: 2.1064 Acc: 0.2780
val Loss: 2.0831 Acc: 0.3097

Epoch 3/24
----------
train Loss: 1.9610 Acc: 0.3451
val Loss: 2.1547 Acc: 0.3004

Epoch 4/24
----------
train Loss: 1.7148 Acc: 0.4179
val Loss: 1.1570 Acc: 0.6026

Epoch 5/24
----------
train Loss: 1.2118 Acc: 0.6082
val Loss: 0.7827 Acc: 0.7220

Epoch 6/24
----------
train Loss: 0.6872 Acc: 0.8209
val Loss: 0.1137 Acc: 0.9869

Epoch 7/24
----------
train Loss: 0.3649 Acc: 0.9310
val Loss: 0.0664 Acc: 0.9963

Epoch 8/24
----------
train Loss: 0.3338 Acc: 0.9366
val Loss: 0.0509 Acc: 1.0000

Epoch 9/24
----------
train Loss: 0.2655 Acc: 0.9646
val Loss: 0.0320 Acc: 1.0000

Epoch 10/24
----------
train Loss: 0.2677 Acc: 0.9534
val Loss: 0.0240 Acc: 1.0000

Epoch 11/24
----------
train Loss: 0.2108 Acc: 0.9739
val Loss: 0.0173 Acc: 1.0000

Ep